In [ ]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect("C:/Users/KULLANICI/Desktop/portfolio-backtest-project/data/database.db")

price_df = pd.read_sql_query("SELECT * FROM price", conn)
fin_df = pd.read_sql_query("SELECT * FROM financial", conn)

price_df["date"] = pd.to_datetime(price_df["date"])

# Her tarihin çeyrek sonunu bul (vektörel)
price_df["quarter_end"] = price_df["date"] + pd.offsets.QuarterEnd(0)

fin_df["period_end"] = pd.to_datetime(
    fin_df["period_year"].astype(str) + "-" + fin_df["period_month"].astype(str) + "-01"
) + pd.offsets.MonthEnd(0)

fin_df = fin_df.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
fin_df["net_income_growth_ttm_yoy"] = (fin_df["net_income_ttm"] / fin_df.groupby('company_id')['net_income_ttm'].shift(4)) - 1

# her şirketin son finansalı
last_financials = fin_df.groupby("company_id")["period_end"].max().rename("last_period_end")
price_df = price_df.merge(last_financials, on="company_id", how="left")

# ---- TEK SATIRDA EFFECTIVE_PERIOD_END ----
price_df["effective_period_end"] = np.where(
    price_df["date"] > price_df["last_period_end"],
    price_df["last_period_end"],
    price_df["quarter_end"]
)

# ---- final merge ----
merged_df = price_df.merge(
    fin_df,
    left_on=["company_id", "effective_period_end"],
    right_on=["company_id", "period_end"],
    how="left"
)
merged_df = merged_df.rename(columns={"date":"date_of_price"})
merged_df["pe"] = merged_df["market_cap"] / merged_df["net_income_ttm"]
merged_df["pb"] = merged_df["market_cap"] / merged_df["equity"]
merged_df["ps"] = merged_df["market_cap"] / merged_df["revenue_ttm"]
merged_df["ev/ebitda"] = (merged_df['market_cap'] + merged_df['gross_debt'] - merged_df['cash_and_cash_equivalents']) / merged_df["ebitda_ttm"]
merged_df["dividend_yield"] = merged_df["dividend_ttm"] / merged_df["market_cap"]


merged_df["peg"] = merged_df["pe"] / merged_df["net_income_growth_ttm_yoy"] / 100

conn.close()
merged_df

,price_id,company_id,date_of_price,open,close,high,low,volume,market_cap,quarter_end,...,eps_ttm,dividend_ttm,period_end,net_income_growth_ttm_yoy,pe,pb,ps,ev/ebitda,dividend_yield,peg
0,1,1,2022-01-03,57.201616,59.558060,59.696676,57.109204,13199632,3.913404e+10,2022-03-31,...,NaN,NaN,2022-03-31,NaN,NaN,4.301755,NaN,NaN,NaN,NaN
1,2,2,2022-01-03,19.916645,21.554981,21.594458,19.571212,95392160,3.013920e+10,2022-03-31,...,NaN,NaN,2022-03-31,NaN,NaN,0.286459,NaN,NaN,NaN,NaN
2,3,1,2022-01-04,60.066319,62.561378,62.561378,60.066319,18241409,4.110744e+10,2022-03-31,...,NaN,NaN,2022-03-31,NaN,NaN,4.518678,NaN,NaN,NaN,NaN
3,4,2,2022-01-04,21.614198,22.502453,23.114363,21.476026,139700845,3.146400e+10,2022-03-31,...,NaN,NaN,2022-03-31,NaN,NaN,0.299050,NaN,NaN,NaN,NaN
4,5,1,2022-01-05,63.300652,64.039932,64.409565,62.746195,12350697,4.207896e+10,2022-03-31,...,NaN,NaN,2022-03-31,NaN,NaN,4.625471,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2698,2699,3,2025-08-29,52.599998,51.650002,53.099998,51.250000,637752,1.549500e+10,2025-09-30,...,0.0,0.000000e+00,2025-06-30,1.646983,41.515570,6.083865,9.856622,23.421440,0.000000,0.252070
2699,2700,2,2025-08-29,340.250000,335.000000,340.500000,333.750000,18346545,4.623000e+11,2025-09-30,...,0.0,-4.461000e+09,2025-06-30,-0.275629,3.856226,0.614663,0.487841,5.723581,-0.009650,-0.139906
2700,2701,1,2025-09-01,530.000000,529.500000,530.000000,520.000000,3811537,3.177000e+11,2025-09-30,...,25.0,-1.064288e+10,2025-06-30,-0.413661,20.520054,2.324002,0.515267,12.085593,-0.033500,-0.496060
2701,2702,3,2025-09-01,51.950001,53.650002,53.950001,51.299999,1146381,1.609500e+10,2025-09-30,...,0.0,0.000000e+00,2025-06-30,1.646983,43.123143,6.319446,10.238291,24.334514,0.000000,0.261831


In [5]:
fin_df

,financial_id,company_id,date_of_publish,period_year,period_month,revenue_ttm,gross_profit_ttm,operating_profit_ttm,ebitda_ttm,net_income_ttm,...,fixed_assets,long_term_debt,short_term_debt,gross_debt,net_debt,equity,eps_c,eps_q,eps_ttm,period_end
0,1,1,None,2025,3,None,None,None,None,None,...,1.815128e+11,4.924420e+10,1.044967e+11,4.209025e+10,1.860062e+10,1.329760e+11,5.0,5.0,None,2025-03-31
1,2,1,None,2025,6,None,None,None,None,None,...,1.962433e+11,5.409232e+10,1.046679e+11,4.646131e+10,3.283389e+10,1.367039e+11,9.0,4.0,None,2025-06-30
2,3,2,None,2025,3,None,None,None,None,None,...,1.182270e+12,4.244820e+11,3.930280e+11,5.376700e+11,3.287730e+11,7.174360e+11,0.0,0.0,None,2025-03-31
3,4,2,None,2025,6,None,None,None,None,None,...,1.255513e+12,4.688030e+11,4.316660e+11,5.845920e+11,3.352410e+11,7.521200e+11,0.0,0.0,None,2025-06-30
4,5,3,None,2025,3,None,None,None,None,None,...,1.999390e+09,2.027256e+08,6.032304e+08,4.042353e+08,-1.637763e+08,2.311534e+09,0.0,0.0,None,2025-03-31
5,6,3,None,2025,6,None,None,None,None,None,...,2.164774e+09,2.095200e+08,5.711083e+08,4.038313e+08,-2.487262e+08,2.546901e+09,0.0,0.0,None,2025-06-30


In [29]:
merged_df = merged_df.rename(columns={"date":"date_of_price"})
merged_df["pe"] = merged_df["market_cap"] / merged_df["net_income_ttm"]
merged_df["pb"] = merged_df["market_cap"] / merged_df["equity"]
merged_df["ps"] = merged_df["market_cap"] / merged_df["revenue_ttm"]
merged_df["ev/ebitda"] = (merged_df['market_cap'] + merged_df['gross_debt'] - merged_df['cash_and_cash_equivalents']) / merged_df["ebitda_ttm"]
merged_df["dividend_yield"] = merged_df["dividend_ttm"] / merged_df["market_cap"]

merged_df["peg"] = merged_df["pe"] / merged_df["net_income_growth_ttm_yoy"]

Buradaki temel mantık şöyleydi: 
1 Ocak - 31 Mart arası: 1. Çeyrek verileriyle eşleşsin. 
1 Nisan - 30 Haziran arası: 2. Çeyrek verileriyle eşleşsin. vs vs
Diyelim ki şu an 1 Ağustos ve 3. çeyrek verisi yok o zaman yine 2. çeyrek verisiyle eşleşsin ama burada bir geleceği görme hatası doğuyor. 1 Ocakta biz 1. çeyrek verilerini bilmiyoruz ki. Date of publish bilgisi olsa bu sıkıntı ortadan kalacaktı ancak onun yerine bir varsayıma gidiyorum ve diyorum ki önceki çeyrek verileri sonraki çeyreğin tam ortasında yayınlanıyor. Yani, 1. çeyrek verileri 15 Mayıs'ta yayınlanıyor. 2. çeyrek verileri 15 Temmuz'da yayınlanıyor, 3. çeyrek verileri 15 Ekim'de yayınlanıyor, 4. çeyrek verileri ise bir sonraki senenin 15 şubatında yayınlanıyor. O tarihten sonra bu verilere erişim olabilir. 

Buraya şöyle bir if eklenebilir. Farz edelim şu an tarih 12 Temmuz 2025 ve 2. Çeyrek verisi mevcut, o zaman 2. Çeyrek verisi kullanılmalı. Bu kısım şöyle algoritmalaştırılabilir: eğer bugünün tarihi > (fiyatın quarter_end) - bir quarter ise ve bu quarter (2. çeyrek 25) için bir financial verisi varsa onu kullan yoksa aynı üstteki algoritma çalışsın.

In [40]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect("C:/Users/KULLANICI/Desktop/portfolio-backtest-project/data/database.db")

price_df = pd.read_sql_query("SELECT * FROM price", conn)
fin_df = pd.read_sql_query("SELECT * FROM financial", conn)

price_df["date"] = pd.to_datetime(price_df["date"])
price_df['company_id'] = pd.to_numeric(price_df['company_id'], errors='coerce')
fin_df['company_id'] = pd.to_numeric(fin_df['company_id'], errors='coerce')

price_df['company_id'] = price_df['company_id'].astype(int)
fin_df['company_id'] = fin_df['company_id'].astype(int)

fin_df["period_end"] = pd.to_datetime(
    fin_df["period_year"].astype(str) + "-" + fin_df["period_month"].astype(str) + "-01"
) + pd.offsets.MonthEnd(0)

fin_df = fin_df.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
fin_df["net_income_growth_ttm_yoy"] = (fin_df["net_income_ttm"] / fin_df.groupby('company_id')['net_income_ttm'].shift(4)) - 1

fin_df

,financial_id,company_id,date_of_publish,period_year,period_month,revenue_ttm,gross_profit_ttm,operating_profit_ttm,ebitda_ttm,net_income_ttm,...,short_term_debt,gross_debt,net_debt,equity,eps_c,eps_q,eps_ttm,dividend_ttm,period_end,net_income_growth_ttm_yoy
0,4,1,None,2022,12,2.792529e+11,4.207056e+10,5.801538e+09,1.363232e+10,1.659943e+10,...,4.747106e+10,1.962585e+10,1.271510e+10,5.830735e+10,28.0,20.0,28.0,-3.497643e+09,2022-12-31,NaN
1,1,1,None,2022,3,NaN,NaN,NaN,NaN,NaN,...,2.108796e+10,8.842232e+09,6.496565e+09,9.097227e+09,2.0,2.0,NaN,NaN,2022-03-31,NaN
2,2,1,None,2022,6,NaN,NaN,NaN,NaN,NaN,...,2.544772e+10,9.843746e+09,7.438998e+09,9.005287e+09,5.0,3.0,NaN,NaN,2022-06-30,NaN
3,3,1,None,2022,9,NaN,NaN,NaN,NaN,NaN,...,2.869317e+10,1.074744e+10,7.298744e+09,1.078349e+10,8.0,3.0,NaN,NaN,2022-09-30,NaN
4,8,1,None,2023,12,4.742004e+11,7.461878e+10,5.717514e+09,1.985372e+10,2.229972e+10,...,7.807136e+10,3.225127e+10,2.216161e+10,1.006923e+11,37.0,15.0,37.0,-4.957372e+09,2023-12-31,0.343403
5,5,1,None,2023,3,3.480165e+11,5.163406e+10,4.009803e+09,1.459379e+10,1.650578e+10,...,3.719496e+10,1.454609e+10,1.051510e+10,2.632998e+10,2.0,2.0,28.0,-3.458768e+09,2023-03-31,NaN
6,6,1,None,2023,6,4.153816e+11,6.139850e+10,3.234744e+09,1.638518e+10,1.880439e+10,...,4.244076e+10,1.709096e+10,1.054108e+10,2.615122e+10,9.0,7.0,32.0,-7.736689e+09,2023-06-30,NaN
7,7,1,None,2023,9,5.075231e+11,7.855225e+10,5.554727e+09,2.207018e+10,2.508553e+10,...,5.292282e+10,1.918579e+10,8.850537e+09,3.004308e+10,22.0,13.0,42.0,-8.186767e+09,2023-09-30,NaN
8,12,1,None,2024,12,6.061988e+11,1.059599e+11,6.185733e+09,2.631918e+10,2.171646e+10,...,8.712409e+10,4.314134e+10,3.352781e+10,1.383038e+11,36.0,13.0,36.0,-8.374857e+09,2024-12-31,-0.026156
9,9,1,None,2024,3,5.214310e+11,8.464737e+10,7.364872e+09,2.273218e+10,2.640523e+10,...,7.445325e+10,2.698775e+10,1.754948e+10,8.417579e+10,9.0,9.0,44.0,-4.957372e+09,2024-03-31,0.599757


In [ ]:
price_df["effective_quarter_end"] = (price_df["date"] - pd.Timedelta(days=45)) + pd.offsets.QuarterEnd(-1)
print(price_df.to_string())

      price_id  company_id       date        open       close        high         low     volume    market_cap effective_quarter_end
0            1           1 2022-01-03   57.201616   59.558060   59.696676   57.109204   13199632  3.913404e+10            2021-09-30
1            2           2 2022-01-03   19.916645   21.554981   21.594458   19.571212   95392160  3.013920e+10            2021-09-30
2            3           1 2022-01-04   60.066319   62.561378   62.561378   60.066319   18241409  4.110744e+10            2021-09-30
3            4           2 2022-01-04   21.614198   22.502453   23.114363   21.476026  139700845  3.146400e+10            2021-09-30
4            5           1 2022-01-05   63.300652   64.039932   64.409565   62.746195   12350697  4.207896e+10            2021-09-30
5            6           2 2022-01-05   21.811588   23.588097   23.805227   21.811588  124793460  3.298200e+10            2021-09-30
6            7           1 2022-01-06   63.300655   63.716496   64.31

In [72]:
import sqlite3
import pandas as pd
import numpy as np
conn = sqlite3.connect("C:/Users/KULLANICI/Desktop/portfolio-backtest-project/data/database.db")

price_df = pd.read_sql_query("SELECT * FROM price", conn)
fin_df = pd.read_sql_query("SELECT * FROM financial", conn)

price_df["date"] = pd.to_datetime(price_df["date"])

price_df["effective_quarter_end"] = (price_df["date"] - pd.Timedelta(days=45)) + pd.offsets.QuarterEnd(-1)

fin_df["period_end"] = pd.to_datetime(
    fin_df["period_year"].astype(str) + "-" + fin_df["period_month"].astype(str) + "-01"
) + pd.offsets.MonthEnd(0)

fin_df = fin_df.sort_values(by=['company_id', 'period_year', 'period_month']).reset_index(drop=True)
fin_df["net_income_growth_ttm_yoy"] = (fin_df["net_income_ttm"] / fin_df.groupby('company_id')['net_income_ttm'].shift(4)) - 1

merged_df = price_df.merge(
    fin_df,
    left_on=["company_id", "effective_quarter_end"],
    right_on=["company_id", "period_end"],
    how="left"
)

merged_df = merged_df.rename(columns={"date":"date_of_price"})
merged_df["pe"] = merged_df["market_cap"] / merged_df["net_income_ttm"]
merged_df["pb"] = merged_df["market_cap"] / merged_df["equity"]
merged_df["ps"] = merged_df["market_cap"] / merged_df["revenue_ttm"]
merged_df["ev/ebitda"] = (merged_df['market_cap'] + merged_df['gross_debt'] - merged_df['cash_and_cash_equivalents']) / merged_df["ebitda_ttm"]
merged_df["dividend_yield"] = merged_df["dividend_ttm"] / merged_df["market_cap"]


merged_df["peg"] = merged_df["pe"] / merged_df["net_income_growth_ttm_yoy"] / 100


merged_df = merged_df[["company_id","date_of_price","period_year","period_month","pe","pb","ps","ev/ebitda","dividend_yield","peg"]]

merged_df.dropna(subset=['period_month'], inplace=True)

conn.close()
merged_df

,company_id,date_of_price,period_year,period_month,pe,pb,ps,ev/ebitda,dividend_yield,peg
233,1,2022-05-16,2022,3,NaN,5.459791,NaN,NaN,NaN,NaN
234,3,2022-05-16,2022,3,NaN,2.509401,NaN,NaN,NaN,NaN
235,2,2022-05-16,2022,3,NaN,0.560326,NaN,NaN,NaN,NaN
236,1,2022-05-17,2022,3,NaN,5.616644,NaN,NaN,NaN,NaN
237,3,2022-05-17,2022,3,NaN,2.473438,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2701,3,2025-09-01,2025,6,43.123143,6.319446,10.238291,24.334514,0.000000,0.261831
2702,2,2025-09-01,2025,6,3.885003,0.619250,0.491481,5.744408,0.009578,-0.140950
2703,1,2025-09-02,2025,6,19.744981,2.236221,0.495805,11.684123,0.034815,-0.477323
2704,3,2025-09-02,2025,6,41.395001,6.066197,9.827996,23.352959,0.000000,0.251338


In [70]:
merged_df.dtypes

company_id                 int64
date_of_price     datetime64[ns]
period_year               object
period_month              object
pe                       float64
pb                       float64
ps                       float64
ev/ebitda                float64
dividend_yield           float64
peg                      float64
dtype: object